## Apartado 1

En este primer ejercicio se implementa la función *string addPunctuationBasic(string)* que, dado un string de entrada que contiene palabras en minísculas y sin signos de puntuación, devuelve dicho string con la primera letra en mayúsculas y un punto al final. Para mayor generalidad, consideramos que la frase pueda contener espacios o tabulaciones al final, y si ya contiene un punto final no añadimos otro.


In [6]:
def addPunctuationBasic(sent):
    sent = sent[0].upper() + sent[1:]
    while sent[-1] == ' ' or sent[-1] == '\t' or sent[-1] == '\n':
        sent = sent[0:-1]
    if sent[-1] == '.' or sent[-1] == ',' or sent[-1] == '.' or sent[-1] == ':' or sent[-1] == ';' or sent[-1] == '?' or sent[-1] == '!':
        return sent
    else:
        return sent + '.'

addPunctuationBasic("i'm Francisco Javier Gañán")

"I'm Francisco Javier Gañán."

## Apartado 2

En este segundo apartado se implementa la función *[(pos, err)] verifyPunctuation(string test, string check)*. Esta función recibe como entrada dos strings y, basándose en la distancia de Levenshtein y en recorrer la matriz que se genera, calcula una lista de mínimas ediciones para convertir *test* en *check*. Se puede encontrar información sobre este algoritmo para calcular la distancia de Levenshtein y la matriz asociada en https://sites.google.com/site/algoritmossimilaridaddistancia/distancia-de-levenshtein. 

En el modelo original de Levenshtein, que es el se usa en este apartado, cada operación de edición (inserción, sustitución o eliminación) tiene coste 1. Por tanto, la distancia de Levenshtein es la suma del número de ediciones realizadas. Needleman y Wunsch, en 1970, lo modificaron para permitir operaciones de edición con distinto costo. Sin embargo, esto no se considera en este ejercicio, ya que, en general, debería asignarse un costo personalizado a cada pareja de palabras para sustitución, eliminación e inserción.

### Tokenizar

La siguiente función (*Tokenize*) se utiliza para tokenizar. Funciona tanto para frases individuales como para textos completos con numerosas frases. Se ha diseñado así para poder tokenizar correctamente los corpus utilizados en este ejercicio.

In [7]:
def Tokenize(seq):
        seqToken = [] # Guarda la secuencia tokenizada
        ant = 0 # Guarda la posición del carácter desde el que comienza un nuevo token
        punt_tuple = (".", ",", ":", ";", "?", "!")
        for i in range(len(seq)):
            if ant <= i:
                if seq[i] == ' ' and not (seq[i-1] in punt_tuple or seq[i-1] == ' ' or seq[i-1] == '\n'):    
                    seqToken.append(seq[ant:i])
                    ant = i + 1
                elif seq[i] == ' ':
                    ant = i + 1
                elif seq[i] in punt_tuple:
                    if ant < i:
                        seqToken.append(seq[ant:i])
                        seqToken.append(seq[i])
                        ant = i + 1
                    else:
                        seqToken.append(seq[i])
                        ant = i + 1
                elif i == len(seq) - 1 and seq[i] != '\n':
                    seqToken.append(seq[ant:i+1])

                elif seq[i].isnumeric():
                    j = i
                    while j < len(seq) and seq[j] != " ": #seq[j] in punt_tuple or seq[j].isnumeric():
                        j += 1
                    if seq[j-1] not in punt_tuple:
                        seqToken.append("$NUMERIC")
                    else:
                        seqToken.append("$NUMERIC")
                        seqToken.append(seq[j-1])
                    ant = j + 1
                # Para leer textos completos considerando el salto de línea. El salto de línea no se considera como token.
                # También se considera que el salto de línea se situa después de un signo de puntuación o de un espacio.
                if seq[i] == "\n":
                    ant = i + 1
                
        return seqToken

### Verificación de la puntuación

En la función *verifyPunctuation* se incluyen 3 funciones, que se ejecutan de forma consecutiva:

- **Tokenize**: La presentada en la celda anterior, para tokenizar los *strings* de entrada.

- **matrizLevenshtein**: Calcula la matriz cuyo último elemento (elemento de coordenadas la última fila y la última columna) es la distancia de Levenshtein. Esta matriz se utiliza para calcular posteriormente la lista de ediciones. 

- **ediciones**: Calcula la lista de ediciones recorriendo la matriz generada previamente, llamda **M**. **M** se recorre empezando en el último elemento. En cada iteración, mientras *i* y *j* (índices de la matriz de filas y columnas, respectivamente) no sean ambos iguales a 0:
    1. Se mira en la vecindad, donde *vecinos* = [**M**(*i -1*, *j*), **M**(*i*, *j - 1*), **M**(*i - 1*, *j - 1*)].
    2. Si el valor de algún vecino es igual a **M**(*i*,*j*), se actualizan *j = j - 1* y *i = i - 1*.
       Si no, se selecciona el valor mínimo de todos los vecinos, y se añade a la lista de ediciones la operación correspondiente (en función de las coordenadas del vecino de valor mínimo):

       - ( *i -1*, *j* ) -> ( 'D',*i* ): Se añade una eliminación a la lista de ediciones. Dicha eliminación se incluye en la lista junto con el índice *i*, que se corresponde con el *string* de *check* en este trabajo. 
       - ( *i*, *j - 1* ) -> ( 'I',*i* ): Se añade una inserción. 
       - ( *i -1*, *j - 1* ) -> ( 'S',*i* ): Se añade una sustitución.

       
       Finalmente, se actualizan *i* y *j* con las coordenadas del vecino seleccionado.

Es importante señalar que este algoritmo genera una de las posibles listas de edición que se pueden conseguir, aunque siempre una de distancia mínima. Esto es porque, al recorrer la matriz, si varios vecinos tienen el mismo valor en el paso 2, se ofrece da la posibilidad de escoger varias secuencias de distancia mínima. El algoritmo escogido para este ejercicio da siempre la misma lista de ediciones para dos secuencias iguales.
        

Inspiración para **matrizLevenshtein**:
https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/

Inspiración para **ediciones**:
https://gist.github.com/curzona/9435822

In [8]:
import numpy as np
def verifyPunctuation(test, check):
    ## Primero se tokenizan las dos secuencias mediante la función Tokenize
    checkToken = Tokenize(check)
    testToken = Tokenize(test)
    punt_tuple = (".", ",", ":", ";", "?", "!")
            
    ## A continuación, se calcula la matriz que computa la distancia de Levenshtein
    def matrizLevenshtein(token1, token2):
        matrix = np.zeros((len(token1) + 1, len(token2) + 1)) # Se crea la matriz
        
        for t1 in range(len(token1) + 1): # La primera fila contiene los índices de las posiciones del token 1
            matrix[t1][0] = t1
        
        for t2 in range(len(token2) + 1): # La primera columna contiene los índices de las posiciones del token 2
            matrix[0][t2] = t2
        
        for t1 in range(1, len(token1) + 1):
            for t2 in range(1, len(token2) + 1):
                # Calcuate 
                if (token1[t1-1] == token2[t2-1]):
                    matrix[t1][t2] = matrix[t1 - 1][t2 - 1]
                else:
                    a = matrix[t1][t2 - 1]
                    b = matrix[t1 - 1][t2]
                    c = matrix[t1 - 1][t2 - 1]

                    if (a <= b and a <= c):
                        matrix[t1][t2] = a + 1
                    elif (b <= a and b <= c):
                        matrix[t1][t2] = b + 1
                    else:
                        matrix[t1][t2] = c + 1 

        return matrix
    matrix_L = matrizLevenshtein(checkToken,testToken)
    ## Se calcula la lista de ediciones
    def ediciones(token1, token2, mat):
        i,j = len(token1), len(token2)
        ediciones = []

        while(not (i==0 and j==0)):
            p = mat[i][j]
            vecinos = []

            if (i!=0 and j!=0):
                vecinos.append(mat[i-1][j-1])

            if (i!=0):
                vecinos.append(mat[i-1][j])
            
            if (j!=0):
                vecinos.append(mat[i][j-1])
            
            min_c = min(vecinos)

            if(min_c == p): # No se añadie ninguna edición
                i, j = i-1, j-1

            elif vecinos.count(min_c) > 1: # Si hay más de una posibilidad en los cambios
                if ( (token1[i-1] not in punt_tuple and token2[j-1] not in punt_tuple) or (token1[i-1] in punt_tuple and token2[j-1] in punt_tuple)) and i!=0 and j!=0:
                    i, j = i-1, j-1
                    ediciones.append(('S', i))
                elif token1[i-1] in punt_tuple and token2[j-1] not in punt_tuple and i!=0:
                    i, j = i-1, j
                    ediciones.append(('D', i))
                elif token1[i-1] not in punt_tuple and token2[j-1] in punt_tuple and j!= 0:
                    i, j = i, j-1
                    ediciones.append(('I', i))
                else:
                    # print("Error")
                    # print(vecinos)
                    # print(p)
                    # print(ediciones)
                    # print(check)
                    # print(test)
                    # print(token1[i])
                    # print(token2[j-1])
                    return 0

            elif(j!=0 and min_c == mat[i][j-1]): # Si solo hay una posibilidad en los cambios, se selecciona
                i, j = i, j-1
                ediciones.append(('I', i))
            elif(i!=0 and j!=0 and min_c == mat[i-1][j-1]):
                i, j = i-1, j-1
                ediciones.append(('S', i))
            elif(i!=0 and min_c == mat[i-1][j]):
                i, j = i-1, j
                ediciones.append(('D', i))

        ediciones.reverse() # Como se ha recorrido la matriz a la inversa, se debe invertir la lista para que esté en el orden adecuado
        return ediciones

 
    return ediciones(checkToken, testToken, matrix_L) , testToken, checkToken

In [9]:
check = "Okay, well, the issue then is, do we need to be alive to see this kind of spontaneous order, and I've already hinted that the answer is no. "
test = "Okay well the issue then, Is do we need to be alive. To see this kind of spontaneous order and i've already hinted that the answer is no."
# check = "Hello. What’s your name?"
# test = "Hello what’s, Your, name?"

# check  = "He had the expectations of his senatorial father and Washington, D.C."
# test = "He had the expectations of his senatorial father and washington dc."
ediciones, b, c = verifyPunctuation(test, check)

print(ediciones)
print(test)
print(check)

[('D', 1), ('D', 3), ('I', 7), ('S', 7), ('D', 8), ('I', 15), ('S', 15), ('D', 22), ('S', 24)]
Okay well the issue then, Is do we need to be alive. To see this kind of spontaneous order and i've already hinted that the answer is no.
Okay, well, the issue then is, do we need to be alive to see this kind of spontaneous order, and I've already hinted that the answer is no. 


A continuación, se comprueba que la función genera una lista de ediciones adecuada.

In [83]:
edit,a,b = verifyPunctuation("Hello, I am. Francisco and. You.", "Hello, I am Francisco, and you?")
print(edit)

# edit = verifyPunctuation("And? If so how do you explain it.", "And, if so, how, do you explain it?")
# print(edit)

[('I', 4), ('D', 5), ('I', 7), ('S', 7), ('S', 8)]


## Apartado 3
 En este apartado se implementa una función (*calculaMetricas*) que calcula el rendimiento de un algoritmo de puntuación, evaluando su resultado en un corpus sobre el que se itera frase a frase. Las métricas que genera son Precisión, *Recall* y F1. También se prueba dicha función para el algoritmo básico de puntuación del apartado 1 sobre el corpus de test.

Lo primero que se hace es leer los corpus de *test* y *check* línea a línea.

In [84]:
## Guardamos los textos de test y check por líneas
f_test = open("datasets/PunctuationTask.test.en","r")
f_check = open("datasets/PunctuationTask.check.en","r")
linesTest = f_test.readlines()
linesCheck = f_check.readlines()
f_test.close()
f_check.close()

A continuación, se calculan las métricas correspondientes. Para ello, es necesario definir los conceptos de TP (*True Positives*), FP (*False Positives*) y FN (*False Negatives*) para un modelo de puntuación. Definimos:

-- **TP**: Cambios correctos que hace el modelo. Estos pueden ser, en general, para los modelos de puntuación que consideramos: Poner mayúscula, poner punto, poner coma, poner dos puntos, poner punto y coma, poner interrogación, poner exclamación. Es decir, añade el signo de puntuación correcto o la mayúscula donde debía realizarse dicho cambio.

-- **FP**: Cambios incorrectos que hace el modelo cuando no se debía hacer un cambio.

-- **FN**: Cambios que omite el modelo que deberían haberse realizado.

**NOTA**: Es importante tener en cuenta que, con estas definiciones, si el modelo realiza un cambio erróneo donde había que realizar un cambio, este cambio no se categoriza como **FP**, ni como **TP**. Esto parece un buen criterio, ya que no afecta tan negativamente al resultado del modelo como un **FP** (pues no empeora la distancia de Levenshtein). 

Por ejemplo: Si el *string* check es "*Qué buen día hace!*" y el *string* test es "qué buen día hace", realizar el cambio "qué buen día hace." no se considera como un **TP**, pues no ha realizado el cambio correcto. Sin embargo, tampoco se considera un **FP**, pues debía realizarse un cambio en esa posición. Contrariamente, si el cambio realizado fuese "qué buen día. hace", este supondría un **FP**, ya que se ha insertado un cambio incorrecto en una posición en la que no se debía realizar ningún cambio.

Para calcular los TP, FP, FN, se sigue el razonamiento ilustrado en la figura.

Si se observan los bloques verdes, estos se corresponden *strings*: con la frase de test original, la frase de check, y la frase que se obtiene al aplicarle el modelo de puntuación. Los bloques azules se corresponden con la distancia de Levenstein entre pares de *strings*. 

Los bloques *verifyPunctuation()* reciben como entrada dos *strings* y devuelven la lista de edición entre ambos. Los bloques *len()* devuelven la longitud de la lista que reciben como entrada, y el bloque *model()* genera los signos de puntuación sobre el *string* de entrada.

- Si se compara el *string* de test con el *string* de Check, se obtiene el número de cambios totales que debe hacer el modelo, esto es **Db = TP + FN**. 

- Si se compara el *string* de test con el *string* del modelo, se obtiene el número de cambios totales que hace el modelo, esto es **Dm = TP + FP**. 

- Si se compara el *string* del modelo con el *string* de Check, ocurre que:
  - **Dcheck** será el resultado de sumar: 
    1. Los **FN**, pues son los cambios que el modelo no ha detectado, cuyas diferencias se siguen manifestado entre ambos *strings*.
    2. Los **FP**, pues son los cambios que el modelo ha hecho indebidamente, que empeoran la distancia de Levenshtein.

**TP**, **FP** y **FN** son incógnitas y **Dm**, **Db** y **Dcheck** son conocidas, por lo que se tiene un sistema de ecuaciones de tres incógnitas compatible determinado que permite conocer dichas incógnitas.

**NOTA**: Es importante destacar que, si se conoce el número de cambios que realiza el modelo, **Dm**, no es necesatio generar dicho número por comparación de strings. Sin embargo, para hacer esta función de evaluación válida para modelos tipo caja negra, se ha decidido computar **Dm** de esta manera.

![Diagrama explicativo](Diagrama_PLN.drawio.png)

In [11]:
import numpy as np
## Función para calcular las métricas
def calculaMetricas(linesTest, linesCheck, model):
    ## Iniciaizamos los valores de TP, FP, FN y la matriz de confusión
    confusion_m = np.zeros((8,8), dtype=int)
    punt_dict = {
    "m": 0, "M": 1, ".": 2, ",": 3, ":": 4, ";": 5, "?": 6, "!": 7
    }
    TP = [0] * 7
    TP_FP = [0] * 7
    TP_FN = [0] * 7
    for (line_test, line_check) in zip(linesTest,linesCheck):
        # Aplicamos el modelo de puntuacion
        modificado = model(line_test)

        cambios_finales, testToken, checkToken = verifyPunctuation(modificado,line_check)
        # print(cambios_finales)
        pos_finales = [c[1] for c in cambios_finales]
        cambios_necesarios  = verifyPunctuation(line_test,line_check)[0]

        correctos = [c for c in cambios_necesarios if c[1] not in pos_finales]
        # print(cambios_necesarios)
        # print(correctos)
        # print(cambios_finales)
        # print("\n")
        for c in correctos:
            if c[0] == 'S' and checkToken[c[1]][0].isupper():
                confusion_m[punt_dict["M"] , punt_dict["M"]]  += 1 # Se ha colocado una mayúscula correctamente (se entiende que el test solo contiene minúsculas)
            else:
                # print(testToken)
                # print(checkToken)
                # print(cambios_finales)
                if checkToken[c[1]] in punt_dict:
                    confusion_m[ punt_dict[checkToken[c[1]]] , punt_dict[checkToken[c[1]]] ] += 1 # Se ha insertado el signo correctamente
        
        offset = 0 # Porque los cambios están referidos al check
        for c in cambios_finales:
            if c[0] == 'S':

                if checkToken[c[1]][0].isupper(): key_c = "M"
                elif checkToken[c[1]] in punt_dict: key_c = checkToken[c[1]]
                else: key_c = "m"

                if testToken[c[1] + offset][0].isupper(): key_t = "M"
                elif testToken[c[1] + offset] in punt_dict: key_t = testToken[c[1] + offset]
                else: key_t = "m"
                
                if key_t == key_c:
                    continue # No se ha cometido un fallo de puntuación

                confusion_m[ punt_dict[key_c] , punt_dict[key_t] ] += 1 #[Valor real , Valor predicho]

            elif c[0] == 'D':
                if checkToken[c[1]] in punt_dict: # Se pone el if ya que en algunos casos la edición real no es la de distancia mínima
                    confusion_m[ punt_dict[checkToken[c[1]]] , punt_dict["m"] ] += 1 #[Valor real , Valor predicho]
                offset -= 1
                
            elif c[0] == 'I':
                if testToken[c[1] + offset] in punt_dict:
                    confusion_m[ punt_dict["m"] , punt_dict[testToken[c[1] + offset]] ] += 1 #[Valor real , Valor predicho]
                offset += 1

    for i in range(len(TP)):
        TP[i] = confusion_m[i+1][i+1]
        TP_FP[i] = sum(confusion_m[:,i+1])
        TP_FN[i] = sum(confusion_m[i+1,:])
        
    TP_total = sum(TP)
    TP_FP_total = sum(TP_FP)
    TP_FN_total = sum(TP_FN)
    
    return confusion_m, TP, TP_FP, TP_FN, TP_total, TP_FP_total, TP_FN_total

Finalmente, mediante la función anterior, se calcula la precisión del modelo básico sobre el conjunto de test.

In [14]:
confusion_matrix, TP, TP_FP, TP_FN, TP_total, TP_FP_total, TP_FN_total = calculaMetricas(linesTest, linesCheck, addPunctuationBasic)
punt_dict = {"m": 0, "M": 1, ".": 2, ",": 3, ":": 4, ";": 5, "?": 6, "!": 7}

def get_key(val):
    for key, value in punt_dict.items():
         if val == value:
             return key

for i , (tp, tp_fp, tp_fn) in enumerate(zip(TP, TP_FP, TP_FN)):
    if tp_fp > 0:
        p = tp/tp_fp
    else:
        p = 0
    if tp_fn > 0:
        r = tp/tp_fn
    else:
        r = 0
    if not (p == 0 and r == 0):
        f1 = 2 * (p*r) / (p+r)
    else:
        f1 = 0

    if tp_fp > 0:
        print("Precision para la operación añadir (" + str(get_key(i+1)) + ") -> " + str(p))
    else:
        print("Este modelo no ha añadido este símbolo")

    if tp_fn > 0:
        print("Recall para la operación añadir (" + str(get_key(i+1)) + ") -> " + str(r))
    else:
        print("En este corpus no aparece este símbolo")

    print("F1 para la operación añadir (" + str(get_key(i+1)) + ") -> " + str(f1))
    print("--------------------------------------------------------------------------")

if TP_FP_total > 0:
    P = TP_total/TP_FP_total
    print("Precision total -> " + str(P))

if tp_fn > 0:
    R = TP_total/TP_FN_total
    print("Recall total -> " + str(R))

print("F1 total -> " + str(2 * (P*R)/(P+R)))
print("--------------------------------------------------------------------------")


NameError: name 'linesTest' is not defined

## Apartado 4

En este apartado se realizan dos tareas: 

- Se crea un modelo de puntuación basado en pseudo-cuatrogramas: Este modelo decidirá, en función de las tres palabras anteriores, si realizar una de las siguientes operaciones: Poner la palabra en mayúscula, poner la palabra en minúscula o añadir un signo de puntuación.

- Una función que, utilizando dicho modelo, realice la operación adecuada.


El modelo se genera a partir de un texto tokenizado, realizando los siguientes pasos:

1. Se crean cuatrogramas de la siguiente forma: (**token1**, **token2**, **token3**, **operación**). Esto se hace recorriendo el texto tokenizado de cuatro en cuatro y, para el último token, sustituir el contenido por la 

In [13]:
def pseudoCuatroGramas(Token):
    # Creamos una lista con los pseudo 4-gramas
    cuatroGramas = []
    for i in range(4, len(Token)):
        c_g = Token[(i - 4):i]   
        if c_g[-1][0].islower():
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 0])
        elif c_g[-1][0].isupper():
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 1])
        elif c_g[-1][0] == '.':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 2])
        elif c_g[-1][0] == ',':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 3])
        elif c_g[-1][0] == ':':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 4])
        elif c_g[-1][0] == ';':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 5])
        elif c_g[-1][0] == '?':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 6])
        elif c_g[-1][0] == '!':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 7])
    # print(cuatroGramas[:200])
    return cuatroGramas
    

In [19]:
f_train = open("datasets/PunctuationTask.train.en","r")
linesTrain = f_train.read() # Guardamos el conjunto de entrenamiento
f_train.close()
trainToken = Tokenize(linesTrain)
cuatroGramas = pseudoCuatroGramas(trainToken)

In [20]:
# Se crea un diccionario para almacenar los tokens únicos y su valor
fDist = dict()
for cg in cuatroGramas:
    if tuple(cg) in fDist:
        fDist[tuple(cg)] += 1
    else:
        fDist[tuple(cg)] = 1
    

In [21]:
## Se crea la función que añade los signos de puntuación
def addPunctuation4gram(sent, addMayusc = True, addDot = True):
    sentToken = Tokenize(sent)

    # Para añadir la primera palabra en mayúscula
    if addMayusc:
        sentToken[0] = sentToken[0].capitalize()

    i = 0
    while i < len(sentToken) - 2:
        valor = tuple(sentToken[i:i + 3])
        max = 0
        max_index = 0
        flag = False
        for j in range(8):
            if tuple([valor, j]) in fDist:
                flag = True
                act = fDist[tuple([valor, j])]
            else:
                continue
            if act > max:
                max = act
                max_index = j
        
        if flag == True:
            # Dependiendo del valor máximo, se eliige una acción u otra
            if max_index == 0 and (i + 3) < len(sentToken) and not sentToken[i+3].islower():
                sentToken[i+3] = sentToken[i+3][0].lower() + sentToken[i+3][1:]
            if max_index == 1 and (i + 3) < len(sentToken):
                sentToken[i+3] = sentToken[i+3].capitalize() 
            elif max_index == 2:
                sentToken.insert(i+3, '.') 
            elif max_index == 3:
                sentToken.insert(i+3, ',') 
            elif max_index == 4:
                sentToken.insert(i+3, ':')    
            elif max_index == 5:
                sentToken.insert(i+3, ';') 
            elif max_index == 6:
                sentToken.insert(i+3, '?') 
            elif max_index == 7:
                sentToken.insert(i+3, '!')
            elif addDot and (i + 3) == len(sentToken) and sentToken[i+2] not in ['.', '?', '!']:
                sentToken.insert(i+3, '.') 
        elif addDot and (i + 3) == len(sentToken) and sentToken[i+2] not in ['.', '?', '!']:
            sentToken.insert(i+3, '.')

        i += 1
    sentReturn = ''
    for k, i in enumerate(sentToken):
        if i == ',' or i == '.' or i == ':' or i == ';' or i == '?' or i == '!' or k == 0:
            sentReturn = sentReturn + i
        else:
            sentReturn = sentReturn + ' ' + i
    return sentReturn

## Apartado 5

In [22]:
confusion_matrix, TP, TP_FP, TP_FN = calculaMetricas(linesTest, linesCheck, addPunctuation4gram)
punt_dict = {"m": 0, "M": 1, ".": 2, ",": 3, ":": 4, ";": 5, "?": 6, "!": 7}

def get_key(val):
    for key, value in punt_dict.items():
         if val == value:
             return key

for i , (tp, tp_fp, tp_fn) in enumerate(zip(TP, TP_FP, TP_FN)):
    if tp_fp > 0:
        p = tp/tp_fp
    else:
        p = 0
    if tp_fn > 0:
        r = tp/tp_fn
    else:
        r = 0
    if not (p == 0 and r == 0):
        f1 = 2 * (p*r) / (p+r)
    else:
        f1 = 0

    if tp_fp > 0:
        print("Precision para la operación añadir (" + str(get_key(i+1)) + ") -> " + str(p))
    else:
        print("Este modelo no ha añadido este símbolo")

    if tp_fn > 0:
        print("Recall para la operación añadir (" + str(get_key(i+1)) + ") -> " + str(r))
    else:
        print("En este corpus no aparece este símbolo")

    print("F1 para la operación añadir (" + str(get_key(i+1)) + ") -> " + str(f1))
    print("--------------------------------------------------------------------------")

Precision para la operación añadir (M) -> 0.8422603865555744
Recall para la operación añadir (M) -> 0.5466853064527997
F1 para la operación añadir (M) -> 0.6630228667038484
--------------------------------------------------------------------------
Precision para la operación añadir (.) -> 0.7986943694683322
Recall para la operación añadir (.) -> 0.8731814438649464
F1 para la operación añadir (.) -> 0.8342785955479789
--------------------------------------------------------------------------
Precision para la operación añadir (,) -> 0.3995123650296064
Recall para la operación añadir (,) -> 0.07046753087178227
F1 para la operación añadir (,) -> 0.11980363484437018
--------------------------------------------------------------------------
Precision para la operación añadir (:) -> 0.14285714285714285
Recall para la operación añadir (:) -> 0.01020408163265306
F1 para la operación añadir (:) -> 0.019047619047619042
--------------------------------------------------------------------------
Pr

In [91]:
TP = 0 
FN = 0
FP = 0

for (line1, line2) in zip(linesTest,linesCheck):
    evaluation = calculaMetricas(line1, line2, addPunctuation4gram)
    TP += evaluation[0]
    FN += evaluation[1]
    FP += evaluation[2]

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * (P * R) / (P + R)

print("Precision: " + str(P))
print("Recall: " + str(R))
print("F1: " + str(F1))

Precision: 0.7728151170281297
Recall: 0.46585981489871203
F1: 0.5813042600444175


In [167]:
# def mix_model(sent):
#     sent2 = addPunctuation4gram(sent)
#     return addPunctuationBasic(sent2)

In [ ]:
# P = 0 
# R = 0
# F1 = 0

# for (line1, line2) in zip(linesTest,linesCheck):
#     evaluation = calculaMetricas(line1, line2, mix_model)
#     P += evaluation[0]
#     R += evaluation[1]
#     F1 += evaluation[2]

# P /= len(linesTest)
# R /= len(linesTest)
# F1 /= len(linesTest)

# print("Precision: " + str(P))
# print("Recall: " + str(R))
# print("F1: " + str(F1))

## Apartado 6

In [6]:
# Cambiamos los textos para que puedan ser leídos por el modelo
# f_test = open("datasets/PunctuationTask.test.en","r")
# f_check = open("datasets/PunctuationTask.check.en","r")
f_train = open("datasets/PunctuationTask.train.en","r")
Test = f_test.read()
Check = f_check.read()
Train = f_train.read()
f_test.close()
f_check.close()
f_train.close()

Check_new = ""
for c in Check:
    if c == ',' or c == '.' or c == ':' or c == ';' or c == '?' or c == '!':
        Check_new += " "
    Check_new += c

Train_new = ""
for c in Train:
    if c == ',' or c == '.' or c == ':' or c == ';' or c == '?' or c == '!':
        Train_new += " "
    Train_new += c


with open('datasets/test.txt', 'w') as f:
    f.write(Test.lower())
f.close()
with open('datasets/dev.txt', 'w') as f:
    f.write(Check_new.lower())
f.close()
with open('datasets/train.txt', 'w') as f:
    f.write(Train_new.lower())
f.close()

In [4]:

!mkdir paraModelo
!python3 Tokenization_Model.py datasets/PunctuationTask.train.en paraModelo/Total.txt

import math

perc_train = 0.8
perc_val = 0.2
f_total = open("paraModelo/Total.txt","r")
total_lines = f_total.readlines()
f_total.close()

trainSize = math.floor(0.8 * len(total_lines))
f = open("paraModelo/train.txt","w")
f.writelines(total_lines[:trainSize])
f.close()

f = open("paraModelo/val.txt", "w")
f.writelines(total_lines[trainSize:])
f.close()

[nltk_data] Downloading package punkt to /home/grvc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Skipped 594 lines


In [15]:
# procesamos el CHECK par pasarselo al modelo y que compruebe
!python3 Tokenization_Check.py datasets/PunctuationTask.check.en paraModelo/check.txt

[nltk_data] Downloading package punkt to /home/grvc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Skipped 62 lines


In [5]:
!python3 Tokenization_Test.py datasets/PunctuationTask.check.en paraModelo/check.txt

[nltk_data] Downloading package punkt to /home/grvc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Skipped 62 lines


In [4]:
from punctuator2tf2FJGO import models, data, main
vocab_len = len(data.read_vocabulary(data.WORD_VOCAB_FILE))
x_len = vocab_len if vocab_len < data.MAX_WORD_VOCABULARY_SIZE else data.MAX_WORD_VOCABULARY_SIZE + data.MIN_WORD_COUNT_IN_VOCAB
x = np.ones((x_len, main.MINIBATCH_SIZE)).astype(int)

print("Loading model parameters...")
net, _ = models.load(model_file, x)

print("Building model...")

word_vocabulary = net.x_vocabulary
punctuation_vocabulary = net.y_vocabulary

reverse_word_vocabulary = {v:k for k,v in word_vocabulary.items()} # Dado un valor, me devuelve su clave (palabra)
reverse_punctuation_vocabulary = {v:k for k,v in punctuation_vocabulary.items()} # Dado un valor, me devuelve su clave (signo de puntuación)

with codecs.open(input_file, 'r', 'utf-8') as f:
    input_text = f.readlines() # read()

if len(input_text) == 0:
    sys.exit("Input file empty.")

text = [line.split() for line in input_text]
# for i,t in enumerate(text):
#     if t not in punctuation_vocabulary and t not in data.PUNCTUATION_MAPPING:
#         pass
#     else:
#         text.pop(i)

#print(text)
#text = [w for w in input_text.split() if w not in punctuation_vocabulary and w not in data.PUNCTUATION_MAPPING] + [data.END]
#text = [w for w in input_text.split() if w not in punctuation_vocabulary and w not in data.PUNCTUATION_MAPPING] + [data.END]
print(restore(text, word_vocabulary, reverse_punctuation_vocabulary, net))

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

# Modelo de puntuación de los autores
def restore(text_lines, word_vocabulary, reverse_punctuation_vocabulary, model):
    i = 0
    puntuated = ''
    text_line = []
    [text_line.append(w) for frase in text_lines for w in frase]
    print(text_line)
    print("Processing line")
    print(text_line)
    if len(text_line) == 0:
        return

    # Si la palabra aparece en el vovabulario, se le pasa a la red, si no, se le pasa el token para caracter desconocido
    converted_subsequence = [word_vocabulary.get(w, word_vocabulary[data.UNK]) for w in text_line]

    # Predicción del modelo
    y = predict(to_array(converted_subsequence), model)
    puntuated = puntuated + (text_line[0])

    last_eos_idx = 0
    punctuations = []
    for y_t in y:

        p_i = np.argmax(tf.reshape(y_t, [-1]))
        punctuation = reverse_punctuation_vocabulary[p_i]

        punctuations.append(punctuation)

        if punctuation in data.EOS_TOKENS:
            last_eos_idx = len(punctuations) # we intentionally want the index of next element
    print(punctuations)
    # if text_line[-1] == data.END:
    #     step = len(text_line) - 1
    # elif last_eos_idx != 0:
    #     step = last_eos_idx
    # else:
    step = len(text_line) - 1

    for j in range(step):
        puntuated = puntuated + (" " + punctuations[j] + " " if punctuations[j] != data.SPACE else " ")
        if j < step - 1:
            puntuated = puntuated + (text_line[1+j])

    puntuated = puntuated + (text_line[-1])

    print(puntuated)
    print("All lines processed")
    return puntuated

def predict(x, model):
    return tf.nn.softmax(net(x))